# Appendix A: Machine Learning Project Checklist

## 1. Overview
**Goal:** While the previous chapters focused on algorithms and code, this notebook focuses on **Process**. This is a comprehensive checklist derived from the book to guide you through any Machine Learning project, ensuring you don't miss critical steps—from framing the problem to deploying the model.

**The 8 Main Steps:**
1.  **Frame the Problem:** Define the objective and look at the big picture.
2.  **Get the Data:** Find, gather, and explore the data.
3.  **Explore the Data:** Gain insights and visualize correlations.
4.  **Prepare the Data:** Clean and transform the data for ML algorithms.
5.  **Shortlist Models:** Train many quick-and-dirty models to find promising candidates.
6.  **Fine-Tune:** Tune hyperparameters and combine models.
7.  **Present:** Document and communicate your solution.
8.  **Launch:** Deploy, monitor, and maintain the system.

**Practical Utility:**
* This notebook includes **Reusable Code Templates** for common tasks like data downloading, structure setup, and correlation analysis.

In [ ]:
# Setup: Standard Imports for any ML Project
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# Best practice: Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Project Environment Setup Complete.")

## Step 1: Frame the Problem

Before touching any code, answer these questions:
1.  **Define the Objective:** What is the business goal? (e.g., Increase revenue? Reduce spam?)
2.  **Current Solution:** How is this problem solved currently? (e.g., Manual review? Simple rules?)
3.  **Type of System:** 
    * Supervised, Unsupervised, or RL?
    * Classification or Regression?
    * Batch or Online learning?
4.  **Performance Measure:** What metric will you use? (RMSE, Accuracy, F1-Score, ROC-AUC?)
5.  **Assumptions:** List all assumptions and verify them if possible.

## Step 2: Get the Data

Automate this step as much as possible so you can get fresh data easily.

**Checklist:**
* List the data you need and how much.
* Find and document where to get that data.
* Check legal obligations (privacy, copyright).
* Create a workspace with enough storage.
* Create a script to download the data.

In [ ]:
# Utility: Generic Data Download Function
import tarfile
import urllib.request

def fetch_data(url, path, filename="data.tgz", extract=True):
    """
    Downloads and optionally extracts a dataset.
    """
    if not os.path.isdir(path):
        os.makedirs(path)
    
    tgz_path = os.path.join(path, filename)
    
    print(f"Downloading from {url}...")
    urllib.request.urlretrieve(url, tgz_path)
    print("Download complete.")
    
    if extract:
        print("Extracting...")
        data_tgz = tarfile.open(tgz_path)
        data_tgz.extractall(path=path)
        data_tgz.close()
        print("Extraction complete.")

# Example usage (commented out)
# fetch_data("https://example.com/data.tgz", "datasets/my_project")

## Step 3: Explore the Data

Try to get insights from a field expert for this step.

**Checklist:**
* Create a copy of the data for exploration (sampling if massive).
* Study each attribute: Name, Type (Categorical/Numerical), % Missing, Noise type, Distribution (Gaussian, Uniform, Log).
* Visualize the data.
* Study correlations between features and the target.
* Identify transformations (e.g., log_transform for heavy tails).

In [ ]:
# Utility: Quick Exploration Helper
def quick_explore(df, target_col=None):
    print("--- Head ---")
    display(df.head())
    
    print("\n--- Info ---")
    print(df.info())
    
    print("\n--- Numerical Stats ---")
    display(df.describe())
    
    print("\n--- Missing Values ---")
    print(df.isnull().sum())
    
    if target_col and target_col in df.columns:
        print(f"\n--- Correlations with {target_col} ---")
        corr_matrix = df.corr(numeric_only=True)
        print(corr_matrix[target_col].sort_values(ascending=False))
        
        print("\n--- Scatter Matrix ---")
        from pandas.plotting import scatter_matrix
        top_features = corr_matrix[target_col].sort_values(ascending=False).head(4).index
        scatter_matrix(df[top_features], figsize=(12, 8))
        plt.show()

## Step 4: Prepare the Data

Work on copies of the data (keep the original clean). Write functions/pipelines.

**Checklist:**
1.  **Data Cleaning:**
    * Fix or remove outliers.
    * Fill missing values (`SimpleImputer`) or drop rows/cols.
2.  **Feature Selection:** Drop attributes that provide no useful information.
3.  **Feature Engineering:**
    * Discretize continuous features.
    * Add promising transformations (log, sqrt, squared).
    * Aggregate features.
4.  **Feature Scaling:** Standardize (`StandardScaler`) or Normalize (`MinMaxScaler`).
5.  **Build a Pipeline:** Use `sklearn.pipeline.Pipeline` to automate this.

## Step 5: Shortlist Promising Models

Train many quick-and-dirty models from different categories (Linear, SVM, Random Forest, Neural Net).

**Checklist:**
* Train models using standard parameters.
* Measure and compare their performance (Use N-fold Cross-Validation).
* Analyze the most significant variables for each algorithm.
* Analyze the types of errors the models make.
* Shortlist the top 3-5 most promising models.

In [ ]:
# Utility: Model Comparison Template
from sklearn.model_selection import cross_val_score

def compare_models(models, X, y, cv=5, scoring='neg_mean_squared_error'):
    results = {}
    for name, model in models.items():
        scores = cross_val_score(model, X, y, cv=cv, scoring=scoring)
        rmse_scores = np.sqrt(-scores)
        results[name] = (rmse_scores.mean(), rmse_scores.std())
        print(f"{name}: Mean={rmse_scores.mean():.4f}, Std={rmse_scores.std():.4f}")
    return results

## Step 6: Fine-Tune the System

You now have a shortlist. It's time to optimize.

**Checklist:**
* **Hyperparameter Tuning:** Use `GridSearchCV` or `RandomizedSearchCV`. Don't manually tweak!
* **Ensemble Methods:** Combine your best models. Voting, Bagging, or Stacking usually performs better than any single model.
* **Evaluate on Test Set:** Once you are confident, run the final model on the test set to estimate the generalization error.
* **Warning:** Do NOT tweak your model after measuring the test error to fix it. You will start overfitting the test set.

## Step 7: Present Your Solution

The technical part is done. Now you must sell it.

**Checklist:**
* Document what you have done.
* Create a nice presentation. Make sure you highlight the big picture first.
* Explain why your solution achieves the business objective.
* Don't forget to present interesting points you noticed along the way (correlations, etc.).
* Describe what worked and what didn't.
* Ensure your key findings are communicated through beautiful visualizations.

## Step 8: Launch!

Get your model ready for production.

**Checklist:**
* **Code Polish:** Ensure code is documented, tested, and follows PEP8.
* **Deployment:** Save model (`joblib` or `SavedModel`). Deploy to REST API (TF Serving) or Cloud (GCP/AWS).
* **Monitoring:** Write monitoring code to check your system's live performance at regular intervals and trigger alerts when it drops.
* **Retraining:** Automate the process of retraining the model on fresh data.
* **Archive:** Archive your models and data versions.